# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconhashav12

In [3]:
path = "../asconhashav12/data/{}.csv"

### Armv7m

In [4]:
with open(path.format("armv7m"), "r") as f:
    aha_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Generazione hash
data = aha_armv7m["0B"]

print_data(data)

0B
69    999
70      1
Name: count, dtype: int64

Massimo: 70
Minimo: 69
Media: 69.001
Varianza: 0.0010000000000000002


#### PT 8 byte

In [6]:
# Generazione hash
data = aha_armv7m["8B"]

print_data(data)

8B
83    800
84    200
Name: count, dtype: int64

Massimo: 84
Minimo: 83
Media: 83.2
Varianza: 0.1601601601601601


#### PT 16 byte

In [7]:
# Generazione hash
data = aha_armv7m["16B"]

print_data(data)

16B
98    798
99    199
97      3
Name: count, dtype: int64

Massimo: 99
Minimo: 97
Media: 98.196
Varianza: 0.16374774774774775


#### PT 32 byte

In [8]:
# Generazione hash
data = aha_armv7m["32B"]

print_data(data)

32B
126    998
127      2
Name: count, dtype: int64

Massimo: 127
Minimo: 126
Media: 126.002
Varianza: 0.001997997997997999


#### PT 64 byte

In [9]:
# Generazione hash
data = aha_armv7m["64B"]

print_data(data)

64B
183    798
185    200
184      2
Name: count, dtype: int64

Massimo: 185
Minimo: 183
Media: 183.402
Varianza: 0.641037037037037


#### PT 128 byte

In [10]:
# Generazione hash
data = aha_armv7m["128B"]

print_data(data)

128B
298    600
299    399
300      1
Name: count, dtype: int64

Massimo: 300
Minimo: 298
Media: 298.401
Varianza: 0.24244144144144134


#### PT 256 byte

In [11]:
# Generazione hash
data = aha_armv7m["256B"]

print_data(data)

256B
528    601
527    398
526      1
Name: count, dtype: int64

Massimo: 528
Minimo: 526
Media: 527.6
Varianza: 0.24224224224224236


#### PT 512 byte

In [12]:
# Generazione hash
data = aha_armv7m["512B"]

print_data(data)

512B
987    797
986    203
Name: count, dtype: int64

Massimo: 987
Minimo: 986
Media: 986.797
Varianza: 0.1619529529529529


#### PT 1024 byte

In [13]:
# Generazione hash
data = aha_armv7m["1024B"]

print_data(data)

1024B
1903    799
1902    200
1904      1
Name: count, dtype: int64

Massimo: 1904
Minimo: 1902
Media: 1902.801
Varianza: 0.16156056056056056


### Armv7m_lowsize

In [14]:
with open(path.format("armv7m_lowsize"), "r") as f:
    aha_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [15]:
# Generazione hash
data = aha_lowsize["0B"]

print_data(data)

0B
56    999
57      1
Name: count, dtype: int64

Massimo: 57
Minimo: 56
Media: 56.001
Varianza: 0.0010000000000000002


#### PT 8 byte

In [16]:
# Generazione hash
data = aha_lowsize["8B"]

print_data(data)

8B
67    998
68      1
69      1
Name: count, dtype: int64

Massimo: 69
Minimo: 67
Media: 67.003
Varianza: 0.004995995995995996


#### PT 16 byte

In [17]:
# Generazione hash
data = aha_lowsize["16B"]

print_data(data)

16B
79    801
80    199
Name: count, dtype: int64

Massimo: 80
Minimo: 79
Media: 79.199
Varianza: 0.1595585585585586


#### PT 32 byte

In [18]:
# Generazione hash
data = aha_lowsize["32B"]

print_data(data)

32B
102    999
103      1
Name: count, dtype: int64

Massimo: 103
Minimo: 102
Media: 102.001
Varianza: 0.0010000000000000002


#### PT 64 byte

In [19]:
# Generazione hash
data = aha_lowsize["64B"]

print_data(data)

64B
149    996
150      3
151      1
Name: count, dtype: int64

Massimo: 151
Minimo: 149
Media: 149.005
Varianza: 0.006981981981981982


#### PT 128 byte

In [20]:
# Generazione hash
data = aha_lowsize["128B"]

print_data(data)

128B
243    599
244    400
245      1
Name: count, dtype: int64

Massimo: 245
Minimo: 243
Media: 243.402
Varianza: 0.24263863863863866


#### PT 256 byte

In [21]:
# Generazione hash
data = aha_lowsize["256B"]

print_data(data)

256B
430    598
431    401
432      1
Name: count, dtype: int64

Massimo: 432
Minimo: 430
Media: 430.403
Varianza: 0.24283383383383392


#### PT 512 byte

In [22]:
# Generazione hash
data = aha_lowsize["512B"]

print_data(data)

512B
805    799
804    199
806      2
Name: count, dtype: int64

Massimo: 806
Minimo: 804
Media: 804.803
Varianza: 0.16235335335335332


#### PT 1024 byte

In [23]:
# Generazione hash
data = aha_lowsize["1024B"]

print_data(data)

1024B
1555    597
1554    400
1556      2
1557      1
Name: count, dtype: int64

Massimo: 1557
Minimo: 1554
Media: 1554.604
Varianza: 0.24943343343343338


### Armv7m_small

In [24]:
with open(path.format("armv7m_small"), "r") as f:
    aha_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [25]:
# Generazione hash
data = aha_small["0B"]

print_data(data)

0B
56    997
55      2
57      1
Name: count, dtype: int64

Massimo: 57
Minimo: 55
Media: 55.999
Varianza: 0.003002002002002001


#### PT 8 byte

In [26]:
# Generazione hash
data = aha_small["8B"]

print_data(data)

8B
67    997
68      2
69      1
Name: count, dtype: int64

Massimo: 69
Minimo: 67
Media: 67.004
Varianza: 0.005989989989989988


#### PT 16 byte

In [27]:
# Generazione hash
data = aha_small["16B"]

print_data(data)

16B
79    601
80    399
Name: count, dtype: int64

Massimo: 80
Minimo: 79
Media: 79.399
Varianza: 0.24003903903903911


#### PT 32 byte

In [28]:
# Generazione hash
data = aha_small["32B"]

print_data(data)

32B
102    799
103    201
Name: count, dtype: int64

Massimo: 103
Minimo: 102
Media: 102.201
Varianza: 0.16075975975975976


#### PT 64 byte

In [29]:
# Generazione hash
data = aha_small["64B"]

print_data(data)

64B
149    999
151      1
Name: count, dtype: int64

Massimo: 151
Minimo: 149
Media: 149.002
Varianza: 0.004000000000000001


#### PT 128 byte

In [30]:
# Generazione hash
data = aha_small["128B"]

print_data(data)

128B
243    598
244    400
242      1
245      1
Name: count, dtype: int64

Massimo: 245
Minimo: 242
Media: 243.401
Varianza: 0.24444344344344335


#### PT 256 byte

In [31]:
# Generazione hash
data = aha_small["256B"]

print_data(data)

256B
430    597
431    402
429      1
Name: count, dtype: int64

Massimo: 431
Minimo: 429
Media: 430.401
Varianza: 0.24244144144144134


#### PT 512 byte

In [32]:
# Generazione hash
data = aha_small["512B"]

print_data(data)

512B
805    799
804    199
806      2
Name: count, dtype: int64

Massimo: 806
Minimo: 804
Media: 804.803
Varianza: 0.16235335335335332


#### PT 1024 byte

In [33]:
# Generazione hash
data = aha_small["1024B"]

print_data(data)

1024B
1555    598
1556    201
1554    200
1557      1
Name: count, dtype: int64

Massimo: 1557
Minimo: 1554
Media: 1555.003
Varianza: 0.40539639639639635


### Bi32_armv7m

In [34]:
with open(path.format("bi32_armv7m"), "r") as f:
    aha_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [35]:
# Generazione hash
data = aha_bi32["0B"]

print_data(data)

0B
56    998
57      1
58      1
Name: count, dtype: int64

Massimo: 58
Minimo: 56
Media: 56.003
Varianza: 0.004995995995995996


#### PT 8 byte

In [36]:
# Generazione hash
data = aha_bi32["8B"]

print_data(data)

8B
67    998
68      2
Name: count, dtype: int64

Massimo: 68
Minimo: 67
Media: 67.002
Varianza: 0.001997997997997999


#### PT 16 byte

In [37]:
# Generazione hash
data = aha_bi32["16B"]

print_data(data)

16B
78    796
79    204
Name: count, dtype: int64

Massimo: 79
Minimo: 78
Media: 78.204
Varianza: 0.16254654654654652


#### PT 32 byte

In [38]:
# Generazione hash
data = aha_bi32["32B"]

print_data(data)

32B
101    999
102      1
Name: count, dtype: int64

Massimo: 102
Minimo: 101
Media: 101.001
Varianza: 0.0010000000000000002


#### PT 64 byte

In [39]:
# Generazione hash
data = aha_bi32["64B"]

print_data(data)

64B
145    598
146    402
Name: count, dtype: int64

Massimo: 146
Minimo: 145
Media: 145.402
Varianza: 0.24063663663663665


#### PT 128 byte

In [40]:
# Generazione hash
data = aha_bi32["128B"]

print_data(data)

128B
235    598
236    401
237      1
Name: count, dtype: int64

Massimo: 237
Minimo: 235
Media: 235.403
Varianza: 0.24283383383383378


#### PT 256 byte

In [41]:
# Generazione hash
data = aha_bi32["256B"]

print_data(data)

256B
414    597
415    400
416      3
Name: count, dtype: int64

Massimo: 416
Minimo: 414
Media: 414.406
Varianza: 0.24741141141141137


#### PT 512 byte

In [42]:
# Generazione hash
data = aha_bi32["512B"]

print_data(data)

512B
774    798
773    200
775      2
Name: count, dtype: int64

Massimo: 775
Minimo: 773
Media: 773.802
Varianza: 0.162958958958959


#### PT 1024 byte

In [43]:
# Generazione hash
data = aha_bi32["1024B"]

print_data(data)

1024B
1492    599
1493    399
1495      1
1494      1
Name: count, dtype: int64

Massimo: 1495
Minimo: 1492
Media: 1492.404
Varianza: 0.2490330330330331


### Ref

In [44]:
with open(path.format("ref"), "r") as f:
    aha_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [45]:
# Generazione hash
data = aha_ref["0B"]

print_data(data)

0B
244    798
245    202
Name: count, dtype: int64

Massimo: 245
Minimo: 244
Media: 244.202
Varianza: 0.16135735735735732


#### PT 8 byte

In [46]:
# Generazione hash
data = aha_ref["8B"]

print_data(data)

8B
287    797
288    202
289      1
Name: count, dtype: int64

Massimo: 289
Minimo: 287
Media: 287.204
Varianza: 0.16454854854854856


#### PT 16 byte

In [47]:
# Generazione hash
data = aha_ref["16B"]

print_data(data)

16B
329    798
330    201
331      1
Name: count, dtype: int64

Massimo: 331
Minimo: 329
Media: 329.203
Varianza: 0.1639549549549549


#### PT 32 byte

In [48]:
# Generazione hash
data = aha_ref["32B"]

print_data(data)

32B
414    597
415    401
416      2
Name: count, dtype: int64

Massimo: 416
Minimo: 414
Media: 414.405
Varianza: 0.24522022022022022


#### PT 64 byte

In [49]:
# Generazione hash
data = aha_ref["64B"]

print_data(data)

64B
585    599
584    398
586      2
587      1
Name: count, dtype: int64

Massimo: 587
Minimo: 584
Media: 584.606
Varianza: 0.24901301301301304


#### PT 128 byte

In [50]:
# Generazione hash
data = aha_ref["128B"]

print_data(data)

128B
926    1000
Name: count, dtype: int64

Massimo: 926
Minimo: 926
Media: 926.0
Varianza: 0.0


#### PT 256 byte

In [51]:
# Generazione hash
data = aha_ref["256B"]

print_data(data)

256B
1606    598
1605    398
1608      2
1607      2
Name: count, dtype: int64

Massimo: 1608
Minimo: 1605
Media: 1605.608
Varianza: 0.25459059059059064


#### PT 512 byte

In [52]:
# Generazione hash
data = aha_ref["512B"]

print_data(data)

512B
2969    998
2968      2
Name: count, dtype: int64

Massimo: 2969
Minimo: 2968
Media: 2968.998
Varianza: 0.0019979979979979972


#### PT 1024 byte

In [53]:
# Generazione hash
data = aha_ref["1024B"]

print_data(data)

1024B
5692    597
5693    400
5694      2
5691      1
Name: count, dtype: int64

Massimo: 5694
Minimo: 5691
Media: 5692.403
Varianza: 0.24683783783783775
